# Notebook for run model

너를 위한 것이란다

## Useful functions

밑에있는 함수들을 사용하렴.. print 대신 print_log를 사용하면 중간에 이 노트북을 꺼도 된단다

In [1]:
def print_log(*args, **kwargs) :
    with open('running_log.log', 'a') as f :
        print(*args, file=f, **kwargs)
    print(*args, **kwargs)

아직 정해진게 아무것도 없다...

### some ideas
1) encoding
- image의 형태는 변하지 않고 위치만 변한다
- 따라서 base image를 추출하면 편함
- Red number는 항상 위에 있으므로 추출 편함
- Green number는 가장 green component가 많은 image에서 추출
- Red number는 좌표를 얻기 편하다
- Green number는 가장 일치하는 점이 많은 곳에서 좌표
- Green number가 어렵다 흠..

2) RNN
- 좌표의 sequence 처리

3) decoding
- 좌표 + base image

## 구현 기준

숫자는 24 by 24 pixel

중심을 기준으로 좌표 설정. 중심은 0~23, 0~23중에서 index를 11로 설정

# Step 0. Import

In [2]:
import os
import numpy as np
import tensorflow as tf

이거 만드는거 개꿀잼

# ㄷㄷ 에반디

# Step 2. RNN

## Configuration of Neural Network

In [3]:
# Training Parameters
learning_rate = 0.001
training_steps = 3000
batch_size = 128
display_step = 200

# Network Parameters
input_channel = 4 # 2 for coordinates of red number, 2 for coordinates of green number
output_channel = 4 # same as input
num_output = 10 # have to predict 10 future frames
timesteps = 19 # timesteps (the number of rows)
num_hidden = 64 # hidden layer num of features
num_layers = 1 # the number of LSTM's layers

## Tensorflow Graph

### Placeholders

In [4]:
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, output_channel, num_output])

NameError: name 'num_input' is not defined

### RNN

In [ ]:
assert num_layers > 0
if num_layers == 1:
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.LSTMBlockCell(num_hidden, forget_bias=1.0)
else:
    cells = [rnn.LSTMBlockCell(num_hidden, forget_bias=1.0) for _ in range (num_layers)]
    lstm_cell = rnn.MultiRNNCell(cells)
    
    
# Get lstm cell output
# time_major=True --> inputs_shape : [timesteps, batch_size, num_hidden]
# time_major=False --> inputs_shape : [batch_size, timesteps, num_hidden]
outputs, states = tf.nn.dynamic_rnn(cell=lstm_cell, inputs=X, time_major=False, dtype=tf.float32)
# outputs : lists consisted of 19 elements. each element has shape of [None, num_hidden]

### Linear regression

As coordinates are results of a regression problem rather than a classification problem, we should use linear regression

In [ ]:
W = tf.Variables(tf.random.uniform([num_hidden, num_output], -1., 1.))
b = tf.Variables(tf.zeros([num_output]))

# Loading data


data shape은 원하는대로 바꾸거라

지금은 red, green 따로해서 input output 있음

In [4]:
data_set_size = 10000
red_train_input = np.zeros([data_set_size, 10, 2], np.uint8)  # data set 크기, sequence 안에서 input은 10개, x좌표 y좌표 해서 2개
red_train_output = np.zeros([data_set_size, 10, 2], np.uint8) #위와 마찬가지
green_train_input = np.zeros([data_set_size, 10, 2], np.uint8)
green_train_output = np.zeros([data_set_size, 10, 2], np.uint8)

red_num_coord_path = "./processed_data/red_num_cordinates/"
green_num_coord_path = "./processed_data/green_num_cordinates/"
for i in range(data_set_size) :
    red_seq_path = os.path.join(red_num_coord_path, '%04d' % i) 
    green_seq_path = os.path.join(green_num_coord_path, '%04d' % i)
    
    with open(red_seq_path) as f :
        for idx in range(10) :
            sp_list = f.readline().split()
            red_train_input[i][idx][0] = int(sp_list[0])
            red_train_input[i][idx][1] = int(sp_list[1])
        for idx in range(10) :
            sp_list = f.readline().split()
            red_train_output[i][idx][0] = int(sp_list[0])
            red_train_output[i][idx][1] = int(sp_list[1])
            
    with open(green_seq_path) as f :
        for idx in range(10) :
            sp_list = f.readline().split()
            green_train_input[i][idx][0] = int(sp_list[0])
            green_train_input[i][idx][1] = int(sp_list[1])
        for idx in range(10) :
            sp_list = f.readline().split()
            green_train_output[i][idx][0] = int(sp_list[0])
            green_train_output[i][idx][1] = int(sp_list[1])

            
# Examples
print(red_train_input[0])
print(red_train_output[0])
print(green_train_input[2031])
print(green_train_output[2031])

# Train these data!

[[16 17]
 [17 15]
 [18 18]
 [19 21]
 [20 24]
 [21 27]
 [22 30]
 [23 33]
 [24 36]
 [25 39]]
[[26 42]
 [27 45]
 [28 46]
 [29 43]
 [30 40]
 [31 37]
 [32 34]
 [33 31]
 [34 28]
 [35 25]]
[[44 36]
 [47 35]
 [48 34]
 [45 33]
 [42 32]
 [39 31]
 [36 30]
 [33 29]
 [30 28]
 [27 27]]
[[24 26]
 [21 25]
 [18 24]
 [17 23]
 [20 22]
 [23 21]
 [26 20]
 [29 19]
 [32 18]
 [35 17]]
